In [138]:
from ase.build import molecule
from ase.io import read, write
import numpy as np

In [8]:
C180 = read("C180-0.xyz")
C60 = read("C60-Ih.xyz")
H20 = molecule("H2O")

write("concat.xyz", C60 + C180 + H20)


In [9]:
!VESTA concat.xyz


(VESTA-gui:30043): Gtk-WARNING **: 14:34:41.589: Negative content width -19 (allocation 1, extents 10x10) while allocating gadget (node button, owner GtkToggleButton)

(VESTA-gui:30043): Gtk-WARNING **: 14:34:41.589: Negative content height -9 (allocation 1, extents 5x5) while allocating gadget (node button, owner GtkToggleButton)


In [10]:
!cat concat.xyz

243
Properties=species:S:1:pos:R:3 C60-Ih=T pbc="F F F"
C        2.16650000       0.59060000       2.58740000
C        3.03780000       0.17660000       1.59180000
C        1.27860000      -0.30980000       3.16790000
C        3.01180000      -1.14340000       1.16540000
C        3.10340000      -1.43350000      -0.19300000
C        3.15030000       1.21060000       0.66820000
C        3.24280000       0.91490000      -0.68590000
C        3.21920000      -0.40230000      -1.12070000
C       -0.43930000       1.35270000       3.12710000
C        0.43630000       2.26180000       2.55420000
C       -0.02960000       0.06330000       3.43790000
C        1.74420000       1.87900000       2.28300000
C        2.35190000       2.26760000       1.09900000
C       -0.26330000       3.02680000       1.63260000
C        0.33740000       3.40540000       0.43730000
C        1.65160000       3.02780000       0.17070000
C       -2.09030000      -0.82250000       2.59550000
C       -2.51110000       

In [139]:
from time import sleep
import numpy as np

def force(r_1, r_2, k=1, eq=1):
    extension = r_2 - r_1
    return -k * (extension - eq)

# def force(x_1, x_2, k=1, eq=1):
#     extension = np.linalg.norm(x_1 - x_2)
#     return -k * (extension - eq)

# def energy(x_1, x_2, k):
#     return force(x_1, x_2, k) * np.hypot(x_1, x_2) / 2

def step(v, x_2, F, a_1, dt):
    v = v + F * dt / 2
    x_2 = x_2 * v * dt
    F = force(x_2, a_1)
    v = v + F * dt / 2
    return v, x_2, F, a_1

# def step(v, x_2, F, a_1, dt):
#     global t
#     v = v + F * dt / 2
#     x_2 = x_2 * v * dt
#     F = force(x_2, a_1)
#     v = v + F * dt / 2
#     return v, x_2, F, a_1

n = 2
r = np.zeroes(shape=(n, 3))
r[0][0] = 2
v = np.zeroes(shape=(n, 3))
F = np.zeroes(shape=(n, 3))

# v = 1
# a_1 = np.array([0.00000000, 0.00000000]).reshape(-1, 1)
# x_2 = np.array([0.00000000, 2.000000000]).reshape(-1, 1)
# F = force(a_1, x_2)
t = 0
dt = 1

with open("output.xyz", "w+") as output:
    print("pos:", x_2)
    print("velocity", float(v))
    print("Force", float(F))


    print(2, file=output)
    print(f"time={t}", file=output)
    print(f"O {float(x_2[0])} {float(x_2[1])} 0.0", file=output)
    print(f"O {float(a_1[0])} {float(a_1[1])} 0.0", file=output)

    for _ in range(100):
        print("pos:", x_2)
        print("velocity", float(v))
        print("Force", float(F))
        v, x_2, F, a_1 = step(v, x_2, F, a_1, dt)
        t += dt

        print(2, file=output)
        print(f"time={t}", file=output)
        print(f"O {float(x_2[0])} {float(x_2[1])} 0.0", file=output)
        print(f"O {float(a_1[0])} {float(a_1[1])} 0.0", file=output)
        
    output.close()


AttributeError: module 'numpy' has no attribute 'zeroes'

In [45]:
!ovito output.xyz

In [124]:
from time import sleep
from ase.build import molecule

def distance(r_j, r_k): return np.linalg.norm(r_j - r_k)

def force(r_j, r_k, k_b=418.39, r_0=1):
    dE_ds = k_b * (distance(r_j, r_k) - r_0)
    ds_dp = (r_j - r_k) / distance(r_j, r_k) 

    # print("force", dE_ds * ds_dp)
    return dE_ds * ds_dp


system = "C3H8"
masses = molecule(system).get_masses()
positions = molecule(system).get_positions()
print(positions)
bonds = [
    [0, 1], [1, 2],  # C-C bonds
    [0, 3], [0, 4], [0, 5],  # H-C bonds for first carbon (C0)
    [1, 6], [1, 7], [1, 8],  # H-C bonds for second carbon (C1)
    [2, 9], [2, 10], [2, 11],  # H-C bonds for third carbon (C2)
]

# positions = np.array([[0, 0, 0], [-1, 1, 0], [1, 1, 0], [0, 0, 3], [-1, 1, 3], [1, 1, 3]], dtype=float)
# masses = np.array([2.0, 1.0, 1.0, 2.0, 1.0, 1.0], dtype=float) / 12
# bonds = [[0, 1], [0, 2]]
velocities = np.zeros_like(positions, dtype=float) # np.random.randn(*positions.shape) 
forces = np.zeros_like(positions, dtype=float)

t = 0
dt = 1e-3

def compute_forces(positions):
    forces = np.zeros_like(positions, dtype=float)
    for j in range(len(forces)):
        for k in range(j + 1, len(forces)):
            if [j, k] not in bonds: 
                # print("skipped", j, k)
                continue

            f_jk = force(positions[j], positions[k])
            # print(f"Force {j} {k} = {f_jk}")

            forces[j] -= f_jk
            # print(f"Forces {j} = {forces[j]}")
            forces[k] += f_jk
    
    return forces

def compute_velocities(velocities, forces, masses):
    for j in range(len(forces)):
        for k in range(j + 1, len(forces)):
            velocities[j] = velocities[j] + forces[j] * dt / 2 / masses[j]
            velocities[k] = velocities[k] + forces[k] * dt / 2 / masses[k]
    

def step(positions, velocities, forces, masses, dt):
    compute_velocities(velocities, forces, masses)

    for j in range(len(positions)):
        for k in range(j + 1, len(positions)):
            positions[j] = positions[j] + velocities[j] * dt
            positions[k] = positions[k] + velocities[k] * dt

    forces = compute_forces(positions)
    compute_velocities(velocities, forces, masses)


    # return positions, velocities, forces
    
def print_frame(positions, time, file):
    print(f"{time=}")
    print(f"{positions=}")
    print(f"{velocities=}")
    print(f"{forces=}")

    n = len(positions)
    print(n, file=file)
    print(f"time={time}", file=file)

    for i in range(n):
        # print(f"H {positions[i][0]} {positions[i][1]} {positions[i][2]}", file=file)
        print(f"{"C" if masses[i] > 12 else "H"} {positions[i][0]} {positions[i][1]} {positions[i][2]}", file=file)



forces = compute_forces(positions)
with open("output.xyz", "w+") as output:
    print_frame(positions, time=t, file=output)

    for _ in range(1000):
        t += dt
        step(positions, velocities, forces, masses, dt)
        print_frame(positions, time=t, file=output)


[[ 0.        0.        0.587716]
 [ 0.        1.266857 -0.260186]
 [ 0.       -1.266857 -0.260186]
 [-0.876898  0.        1.244713]
 [ 0.876898  0.        1.244713]
 [ 0.        2.16615   0.362066]
 [ 0.       -2.16615   0.362066]
 [ 0.883619  1.304234 -0.904405]
 [-0.883619  1.304234 -0.904405]
 [-0.883619 -1.304234 -0.904405]
 [ 0.883619 -1.304234 -0.904405]]
time=0
positions=array([[ 0.      ,  0.      ,  0.587716],
       [ 0.      ,  1.266857, -0.260186],
       [ 0.      , -1.266857, -0.260186],
       [-0.876898,  0.      ,  1.244713],
       [ 0.876898,  0.      ,  1.244713],
       [ 0.      ,  2.16615 ,  0.362066],
       [ 0.      , -2.16615 ,  0.362066],
       [ 0.883619,  1.304234, -0.904405],
       [-0.883619,  1.304234, -0.904405],
       [-0.883619, -1.304234, -0.904405],
       [ 0.883619, -1.304234, -0.904405]])
velocities=array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]

In [151]:
from time import sleep
from ase.build import molecule
from tqdm import tqdm
import numpy as np

def distance(r_j, r_k): return np.linalg.norm(r_j - r_k)

def force(r_j, r_k, k_b=2000, r_0=1.385):
    dE_ds = k_b * (distance(r_j, r_k) - r_0)
    ds_dp = (r_j - r_k) / distance(r_j, r_k) 

    # print("force", dE_ds * ds_dp)
    return dE_ds * ds_dp


system = molecule("C180")
masses = molecule(system).get_masses()
positions = molecule(system).get_positions()
bonds = []
print(positions)
for i in range(len(positions)):
    for j in range(i + 1, len(positions)):
        if distance(positions[i], positions[j]) < 1.5:
            bonds.append([i, j])

print("number of bonds", len(bonds))

# positions = np.array([[0, 0, 0], [-1, 1, 0], [1, 1, 0], [0, 0, 3], [-1, 1, 3], [1, 1, 3]], dtype=float)
# masses = np.array([2.0, 1.0, 1.0, 2.0, 1.0, 1.0], dtype=float) / 12
# bonds = [[0, 1], [0, 2]]
velocities = np.zeros_like(positions, dtype=float) # np.random.randn(*positions.shape) 
forces = np.zeros_like(positions, dtype=float)

t = 0
dt = 1e-4

def compute_forces(positions):
    forces = np.zeros_like(positions, dtype=float)
    for j in range(len(forces)):
        for k in range(j + 1, len(forces)):
            if [j, k] not in bonds: 
                # print("skipped", j, k)
                continue

            f_jk = force(positions[j], positions[k])
            # print(f"Force {j} {k} = {f_jk}")

            forces[j] -= f_jk
            # print(f"Forces {j} = {forces[j]}")
            forces[k] += f_jk
    
    return forces

def compute_velocities(velocities, forces, masses):
    for j in range(len(forces)):
        for k in range(j + 1, len(forces)):
            velocities[j] = velocities[j] + forces[j] * dt / 2 / masses[j]
            velocities[k] = velocities[k] + forces[k] * dt / 2 / masses[k]
    

def step(positions, velocities, forces, masses, dt):
    compute_velocities(velocities, forces, masses)

    for j in range(len(positions)):
        for k in range(j + 1, len(positions)):
            positions[j] = positions[j] + velocities[j] * dt
            positions[k] = positions[k] + velocities[k] * dt

    forces = compute_forces(positions)
    compute_velocities(velocities, forces, masses)


    # return positions, velocities, forces
    
def print_frame(positions, time, file):
    # print(f"{time=}")
    # print(f"{positions=}")
    # print(f"{velocities=}")
    # print(f"{forces=}")

    n = len(positions)
    print(n, file=file)
    print(f"time={time}", file=file)

    for i in range(n):
        # print(f"H {positions[i][0]} {positions[i][1]} {positions[i][2]}", file=file)
        print(f"C {positions[i][0]} {positions[i][1]} {positions[i][2]}", file=file)



forces = compute_forces(positions)
with open("my-c60.xyz", "w+") as output:
    print_frame(positions, time=t, file=output)

    for _ in tqdm(range(2000)):
        t += dt
        step(positions, velocities, forces, masses, dt)
        print_frame(positions, time=t, file=output)


[[ 4.06800035e+00  0.00000000e+00  0.00000000e+00]
 [ 3.82266991e+00  1.39133806e+00  0.00000000e+00]
 [ 3.52299164e+00  2.03400017e+00  1.22975607e+00]
 [ 2.61486022e+00  3.11626906e+00  1.22975607e+00]
 [ 2.03400017e+00  3.52299164e+00 -4.44089210e-16]
 [ 7.06400847e-01  4.00619828e+00  2.45951215e+00]
 [ 2.49093180e-16  4.06800035e+00  1.22975607e+00]
 [-1.39133806e+00  3.82266991e+00  1.22975607e+00]
 [-2.03400017e+00  3.52299164e+00 -8.88178420e-16]
 [-3.11626906e+00  2.61486022e+00  2.45951215e+00]
 [-3.52299164e+00  2.03400017e+00  1.22975607e+00]
 [-4.00619828e+00  7.06400847e-01  1.22975607e+00]
 [-4.06800035e+00  4.98186360e-16 -8.88178420e-16]
 [-3.82266991e+00 -1.39133806e+00  2.45951215e+00]
 [-3.52299164e+00 -2.03400017e+00  1.22975607e+00]
 [-2.61486022e+00 -3.11626906e+00  1.22975607e+00]
 [-2.03400017e+00 -3.52299164e+00 -1.77635684e-15]
 [-7.06400847e-01 -4.00619828e+00  2.45951215e+00]
 [-7.47279540e-16 -4.06800035e+00  1.22975607e+00]
 [ 1.39133806e+00 -3.82266991e+

100%|██████████| 200/200 [02:24<00:00,  1.38it/s]


In [1]:
from time import sleep
from ase.build import molecule, nanotube
from tqdm import tqdm
from math import sqrt
import numpy as np

def distance(r_j, r_k): return np.linalg.norm(r_j - r_k)

def force(r_j, r_k, k_b=1050, r_0=1.6): # r_0=1.385
    dE_ds = k_b * (distance(r_j, r_k) - r_0)
    ds_dp = (r_j - r_k) / distance(r_j, r_k)

    # print("force", dE_ds * ds_dp)
    return dE_ds * ds_dp

def lennard_jones_force(r_j, r_k, epsilon, sigma=1.6):
    r = distance(r_j, r_k)
    x6r7 = sigma**6 * r**-7

    dE_ds = 24 * epsilon * ((-2* x6r7 ** 2) * r + x6r7)
    ds_dp = (r_j - r_k) / distance(r_j, r_k)

    return dE_ds * ds_dp


system = nanotube(6, 6, 10)
masses = system.get_masses()
positions = system.get_positions()
bonds = []
print(positions)
for i in range(len(positions)):
    for j in range(i + 1, len(positions)):
        if distance(positions[i], positions[j]) < 1.5:
            bonds.append([i, j])

print("number of bonds", len(bonds))

# positions = np.array([[0, 0, 0], [-1, 1, 0], [1, 1, 0], [0, 0, 3], [-1, 1, 3], [1, 1, 3]], dtype=float)
# masses = np.array([2.0, 1.0, 1.0, 2.0, 1.0, 1.0], dtype=float) / 12
# bonds = [[0, 1], [0, 2]]
velocities = np.zeros_like(positions, dtype=float) # np.random.randn(*positions.shape) 
forces = np.zeros_like(positions, dtype=float)

t = 0
dt = 1e-4

def compute_forces(positions):
    forces = np.zeros_like(positions, dtype=float)
    for j in range(len(forces)):
        for k in range(j + 1, len(forces)):
            if [j, k] not in bonds: 
                # print("skipped", j, k)
                continue

            f_jk = force(positions[j], positions[k])
            # print(f"Force {j} {k} = {f_jk}")

            forces[j] -= f_jk
            # print(f"Forces {j} = {forces[j]}")
            forces[k] += f_jk
    
    return forces

def compute_velocities(velocities, forces, masses):
    for j in range(len(forces)):
        for k in range(j + 1, len(forces)):
            velocities[j] = velocities[j] + forces[j] * dt / 2 / masses[j]
            velocities[k] = velocities[k] + forces[k] * dt / 2 / masses[k]
    

def step(positions, velocities, forces, masses, dt):
    compute_velocities(velocities, forces, masses)

    for j in range(len(positions)):
        for k in range(j + 1, len(positions)):
            positions[j] = positions[j] + velocities[j] * dt
            positions[k] = positions[k] + velocities[k] * dt

    forces = compute_forces(positions)
    compute_velocities(velocities, forces, masses)


    # return positions, velocities, forces
    
def print_frame(positions, time, file):
    # print(f"{time=}")
    # print(f"{positions=}")
    # print(f"{velocities=}")
    # print(f"{forces=}")

    n = len(positions)
    print(n, file=file)
    print(f"time={time}", file=file)

    for i in range(n):
        # print(f"H {positions[i][0]} {positions[i][1]} {positions[i][2]}", file=file)
        print(f"C {positions[i][0]} {positions[i][1]} {positions[i][2]}", file=file)



forces = compute_forces(positions)
with open("nanotube.xyz", "w+") as output:
    print_frame(positions, time=t, file=output)

    for _ in tqdm(range(400)):
        t += dt
        step(positions, velocities, forces, masses, dt)
        print_frame(positions, time=t, file=output)


[[ 4.06800035e+00  0.00000000e+00  0.00000000e+00]
 [ 3.82266991e+00  1.39133806e+00  0.00000000e+00]
 [ 3.52299164e+00  2.03400017e+00  1.22975607e+00]
 [ 2.61486022e+00  3.11626906e+00  1.22975607e+00]
 [ 2.03400017e+00  3.52299164e+00 -4.44089210e-16]
 [ 7.06400847e-01  4.00619828e+00  2.45951215e+00]
 [ 2.49093180e-16  4.06800035e+00  1.22975607e+00]
 [-1.39133806e+00  3.82266991e+00  1.22975607e+00]
 [-2.03400017e+00  3.52299164e+00 -8.88178420e-16]
 [-3.11626906e+00  2.61486022e+00  2.45951215e+00]
 [-3.52299164e+00  2.03400017e+00  1.22975607e+00]
 [-4.00619828e+00  7.06400847e-01  1.22975607e+00]
 [-4.06800035e+00  4.98186360e-16 -8.88178420e-16]
 [-3.82266991e+00 -1.39133806e+00  2.45951215e+00]
 [-3.52299164e+00 -2.03400017e+00  1.22975607e+00]
 [-2.61486022e+00 -3.11626906e+00  1.22975607e+00]
 [-2.03400017e+00 -3.52299164e+00 -1.77635684e-15]
 [-7.06400847e-01 -4.00619828e+00  2.45951215e+00]
 [-7.47279540e-16 -4.06800035e+00  1.22975607e+00]
 [ 1.39133806e+00 -3.82266991e+

 27%|██▋       | 108/400 [01:38<04:25,  1.10it/s]


KeyboardInterrupt: 

In [2]:
from time import sleep
from ase.build import molecule, nanotube, bulk
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution
from ase.io import write
from ase import Atoms
from tqdm import tqdm
from math import sqrt
from pathlib import Path
import numpy as np

def distance(r_j, r_k): return np.linalg.norm(r_j - r_k)

def force(r_j, r_k, k_b=1050, r_0=1.6): # r_0=1.385
    dE_ds = k_b * (distance(r_j, r_k) - r_0)
    ds_dp = (r_j - r_k) / distance(r_j, r_k)

    # print("force", dE_ds * ds_dp)
    return dE_ds * ds_dp


def lennard_jones_force(r, epsilon=0.01034, sigma=3.405):
    # r = distance(r_j, r_k)
    x6r6 = sigma**6 * r**-6

    dE_ds = 24 * epsilon * ((-2* x6r6 ** 2) + x6r6) / r
    energy = 4 * epsilon * (x6r6 ** 2 - x6r6)

    return (dE_ds, energy)

p = Path("logs.csv")
p.unlink(missing_ok=True)

size = 5.3
temperature = 50 # K

system = bulk("Ar", "fcc", a=size, cubic=True) * (2, 2, 2)
MaxwellBoltzmannDistribution(system, temperature_K=temperature, rng=np.random.RandomState(2024))
write("argon.xyz", system)

n_atoms = len(system)


velocities = system.get_velocities()
forces = np.zeros_like(velocities, dtype=float)

t = 0
# dt = 1e-13
dt = 1e-6

logs = open(f"logs-{dt}.csv", "w+")

def compute_forces(system, n_atoms):
    e_potential = 0.0
    for j in range(n_atoms):
        for k in range(j + 1, n_atoms):
            r_v = system.get_distance(k, j, mic=True, vector=True)
            r = np.linalg.norm(r_v, axis=0)


            if r > 5: continue

            ds_dp = r_v / r

            f_jk, ep = lennard_jones_force(r)
            f_jk *= ds_dp
            e_potential += ep
            # print(f"Force {j} {k} = {f_jk}")

            forces[j] -= f_jk
            # print(f"Forces {j} = {forces[j]}")
            forces[k] += f_jk
    
    return (forces, e_potential)

def compute_velocities(system: Atoms,  forces ):
    velocities = system.get_velocities()
    masses = system.get_masses()
    for j in range(len(forces)):
        for k in range(j + 1, len(forces)):
            velocities[j] = velocities[j] + forces[j] * dt / 2 / masses[j]
            velocities[k] = velocities[k] + forces[k] * dt / 2 / masses[k]
    system.set_velocities(velocities)
    

def step(system: Atoms, n_atoms , forces, dt):
    compute_velocities(system,  forces)

    velocities = system.get_velocities()
    for j in range(n_atoms):
        for k in range(j + 1, n_atoms):
            system[j].position += velocities[j] * dt
            system[k].position += velocities[k] * dt

    forces, e_potential = compute_forces(system, n_atoms)

    compute_velocities(system,  forces)
    system.wrap()

    e_kinetic = system.get_kinetic_energy()

    # print(f"p:{e_potential} + k:{e_kinetic} = {e_potential + e_kinetic}")
    print(f"{t} {e_potential} {e_kinetic} {e_potential + e_kinetic}", file=logs)


    # return positions, velocities, forces
    

forces, energy = compute_forces(system, n_atoms)
e_kinetic = system.get_kinetic_energy()
print("total", energy)
write("crystal.xyz", system, format="extxyz")

for _ in tqdm(range(int(1e5))):
    t += dt
    step(system, n_atoms , forces, dt)
    write("crystal.xyz", system, format="extxyz", append=True)

logs.close()


total -1.954240788640888


  0%|          | 51/100000 [00:07<4:15:48,  6.51it/s]


KeyboardInterrupt: 

In [23]:
from tqdm import tqdm
from math import sqrt
from pathlib import Path
import numpy as np

# Constants
G = 6.67430e-11  # Gravitational constant
dt = 60 * 60 * 24
n_steps = int(1e6)  # Number of simulation steps, should be an integer

planet_names = ["Sun", "Mercury", "Venus", "Earth", "Mars", "Jupiter", "Saturn", "Uranus", "Neptune", "Pluto"]
element_symbols = ["S", "H", "O", "C", "He", "Fe", "Pb", "Si", "P", "H"]

masses = np.array([
    1.989e30,  # Sun
    3.3011e23,  # Mercury
    4.8675e24,  # Venus
    5.97237e24,  # Earth
    6.4171e23,  # Mars
    1.8982e27,  # Jupiter
    5.6834e26,  # Saturn
    8.6810e25,  # Uranus
    1.02413e26,  # Neptune
    1.303e22  # Pluto
])

# Initial positions (x, y, z) in meters, simplified to 2D for this example
positions = np.array([
    [0, 0, 0],  # Sun
    [0.39e11, 0, 0],  # Mercury
    [0.72e11, 0, 0],  # Venus
    [1.0e11, 0, 0],  # Earth
    [1.52e11, 0, 0],  # Mars
    [5.2e11, 0, 0],  # Jupiter
    [9.58e11, 0, 0],  # Saturn
    [1.43e12, 0, 0],  # Uranus
    [2.87e12, 0, 0],   # Neptune
    [5.9e12, 0, 0]   # Pluto
])

# Initial velocities (vx, vy, vz) in meters per second, simplified to 2D velocities
velocities = np.array([
    [0, 0, 0],  # Sun
    [0, 47.87e3, 0],  # Mercury
    [0, 35.02e3, 0],  # Venus
    [0, 29.78e3, 0],  # Earth
    [0, 24.077e3, 0],  # Mars
    [0, 13.07e3, 0],  # Jupiter
    [0, 9.69e3, 0],  # Saturn
    [0, 6.81e3, 0],  # Uranus
    [0, 5.43e3, 0],   # Neptune
    [0, 4.74e3, 0]   # Pluto
])

n_atoms = len(positions)
forces = np.zeros((n_atoms, 3), dtype=float)

def gravitation_force(r, m1, m2):
    return G * m1 * m2 / r**2

def compute_forces():
    global forces
    forces.fill(0.0)
    for j in range(n_atoms):
        for k in range(j + 1, n_atoms):
            r_v = positions[k] - positions[j]
            r = np.linalg.norm(r_v)
            f_mag = gravitation_force(r, masses[j], masses[k])
            f_jk = f_mag * r_v / r  # Directional force
            forces[j] += f_jk
            forces[k] -= f_jk

def compute_velocities():
    global velocities
    velocities += (forces * dt) / masses[:, np.newaxis]

def step():
    global positions, velocities
    compute_velocities()
    positions += velocities * dt
    compute_forces()
    compute_velocities()

scale = 1e-11 * 2
def print_frame(positions, time, file):
    n = len(positions)
    print(n, file=file)
    print(f"time={time}", file=file)

    for i in range(n):
        # if i == 0:
        #     print(f"{"S"} {positions[i][0] * scale} {positions[i][1] * scale} {positions[i][2] * scale}", file=file)
        # else:
        print(f"{element_symbols[i]} {positions[i][0] * scale} {positions[i][1] * scale} {positions[i][2] * scale}", file=file)

p = Path("logs.csv")
p.unlink(missing_ok=True)

t = 0
logs = open(f"logs-{dt}.csv", "w+")
output = open(f"planets.xyz", "w+")

compute_forces()
e_kinetic = 0.5 * np.sum(masses[:, np.newaxis] * velocities**2)

print_frame(positions, time=t, file=output)
for _ in tqdm(range(n_steps)):
    t += dt  # No scaling down of time step
    step()
    print_frame(positions, time=t, file=output)

logs.close()
output.close()

 10%|▉         | 95693/1000000 [00:40<06:26, 2339.11it/s]


KeyboardInterrupt: 

In [22]:

from tqdm import tqdm
from math import sqrt
from pathlib import Path
import numpy as np

# Constants
G = 6.67430e-11  # Gravitational constant
dt = 60
n_steps = int(1e6)  # Number of simulation steps, should be an integer

planet_names = ["Sun", "Earth", "Moon", "Mercury", "Venus", "Mars", "Jupiter", "Saturn", "Uranus", "Neptune", "Pluto"]
element_symbols = ["He", "O", "H", "Hg", "V", "Fe", "H", "S", "H", "N", "P"]

masses = np.array([
    1.989e30,  # Sun
    5.97237e24,  # Earth
    7.347673e22,  # Moon (corrected mass)
    # 3.3011e23,  # Mercury
    # 4.8675e24,  # Venus
    # 6.4171e23,  # Mars
    # 1.8982e27,  # Jupiter
    # 5.6834e26,  # Saturn
    # 8.6810e25,  # Uranus
    # 1.02413e26,  # Neptune
    # 1.303e22  # Pluto
])

# Initial positions (x, y, z) in meters, simplified to 2D for this example
positions = np.array([
    [0, 0, 0], 
    [1.51e11, 0, 0],  #  Earth
    [1.51e11 + 384400e3 , 0, 0], # Moon (corrected position relative to Earth)
    # [0.39e11, 0, 0],  # Mercury
    # [0.72e11, 0, 0],  # Venus
    # [1.52e11, 0, 0],  # Mars
    # [5.2e11, 0, 0],  # Jupiter
    # [9.58e11, 0, 0],  # Saturn
    # [1.43e12, 0, 0],  # Uranus
    # [2.87e12, 0, 0],  # Neptune
    # [5.9e12, 0, 0]   # Pluto
])

# Initial velocities (vx, vy, vz) in meters per second, simplified to 2D velocities
velocities = np.array([
    [0, 0, 0],  # Sun
    [0, 29.78e3, 0],  # Earth
    [0, 30.88e3, 0],  # Moon
    # [0, 47.87e3, 0],  # Mercury
    # [0, 35.02e3, 0],  # Venus
    # [0, 24.077e3, 0],  # Mars
    # [0, 13.07e3, 0],  # Jupiter
    # [0, 9.69e3, 0],  # Saturn
    # [0, 6.81e3, 0],  # Uranus
    # [0, 5.43e3, 0],  # Neptune
    # [0, 4.74e3, 0]   # Pluto
])


n_atoms = len(positions)
forces = np.zeros((n_atoms, 3), dtype=float)

def gravitation_force(r, m1, m2):
    return G * m1 * m2 / r**2

def compute_forces():
    global forces
    forces.fill(0.0)
    for j in range(n_atoms):
        for k in range(j + 1, n_atoms):
            r_v = positions[k] - positions[j]
            r = np.linalg.norm(r_v)
            f_mag = gravitation_force(r, masses[j], masses[k])
            f_jk = f_mag * r_v / r  # Directional force
            forces[j] += f_jk
            forces[k] -= f_jk

def compute_velocities():
    global velocities
    velocities += (forces * dt) / masses[:, np.newaxis]

def step():
    global positions, velocities
    compute_velocities()
    positions += velocities * dt
    compute_forces()
    compute_velocities()

scale = 1e-10 * 2
def print_frame(positions, time, file):
    n = len(positions)
    print(n, file=file)
    print(f"time={time}", file=file)

    for i in range(n):
        # if i == 0:
        #     print(f"{"S"} {positions[i][0] * scale} {positions[i][1] * scale} {positions[i][2] * scale}", file=file)
        # else:
        print(f"{element_symbols[i]} {positions[i][0] * scale} {positions[i][1] * scale} {positions[i][2] * scale}", file=file)
        # print(f"{"C"} {positions[i][0] * scale} {positions[i][1] * scale} {positions[i][2] * scale}", file=file)

p = Path("logs.csv")
p.unlink(missing_ok=True)

t = 0
logs = open(f"logs-{dt}.csv", "w+")
output = open(f"earth-moon.xyz", "w+")

compute_forces()
e_kinetic = 0.5 * np.sum(masses[:, np.newaxis] * velocities**2)

print_frame(positions, time=t, file=output)
for _ in tqdm(range(n_steps)):
    t += dt  # No scaling down of time step
    step()
    print_frame(positions, time=t, file=output)

logs.close()
output.close()

 12%|█▏        | 117419/1000000 [00:07<00:53, 16625.12it/s]


KeyboardInterrupt: 